<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Wyjanienie-modelu---analiza-bdw/colabs/Similarity_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 1926, done.
remote: Counting objects: 100% (547/547), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 1926 (delta 469), reused 415 (delta 368), pack-reused 1379
Receiving objects: 100% (1926/1926), 6.11 MiB | 12.33 MiB/s, done.
Resolving deltas: 100% (1201/1201), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


In [3]:
# !git checkout Wyjanienie-modelu---analiza-bdw

Branch 'Wyjanienie-modelu---analiza-bdw' set up to track remote branch 'Wyjanienie-modelu---analiza-bdw' from 'origin'.
Switched to a new branch 'Wyjanienie-modelu---analiza-bdw'


In [4]:
!chmod +x setup_sparka.sh
!./setup_sparka.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 26.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=e4475a7968c05310831c6752bdab23cc32493d85acd5a5c57ccc5df0f25fdcf5
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2023-01-22 21:55:40--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274099817 

In [5]:
import pandas as pd
from lib.pyspark_init import create_spark_context
from lib.similarity_utils import *
from lib.const import TRAIN_DATA, VALIDATE_DATA, SIMILARITY_VALUES, RANKINGS

In [6]:
spark = create_spark_context()
spark

In [7]:
%%time
train_data = pd.read_parquet(TRAIN_DATA, columns=["nconst", "tconst", "category", "primaryName", "knownForTitles",
                                                  "no_nominations_oscars_norm", "no_nominations_globes_norm",
                                                  "no_nominations_emmy_norm", "no_films_norm",
                                                  "average_films_rating_norm", "genres_code", "types_code"])
validate_data = pd.read_parquet(VALIDATE_DATA, columns=["nconst", "tconst", "category", "primaryName", "knownForTitles",
                                                  "no_nominations_oscars_norm", "no_nominations_globes_norm",
                                                  "no_nominations_emmy_norm", "no_films_norm",
                                                  "average_films_rating_norm", "genres_code", "types_code"])

CPU times: user 33.3 s, sys: 7.24 s, total: 40.5 s
Wall time: 53.3 s


# Porównywanie dwóch aktorów

Wyciągnięcie pierwszego aktora

Weź id aktora ze strony imdb. Wpisujemy imię i nazwisko aktora w wyszukiwarkę na imdb i kopiujemy jego id z linku, np. dla Natalie Portman id bierzemy z linku: https://www.imdb.com/name/nm0000204/?ref_=nv_sr_srsg_0

In [8]:
actor1_id = "nm0000204"

In [9]:
if (train_data['nconst'] == actor1_id).any():
    actor1_data = find_actor(train_data,  actor1_id)
    display(pd.DataFrame(actor1_data))
    actor1_data = prepare_pandas_row(actor1_data)
elif (validate_data['nconst'] == actor1_id).any():
    actor1_data = find_actor(validate_data,  actor1_id)
    display(pd.DataFrame(actor1_data))
    actor1_data = prepare_pandas_row(actor1_data)
else:
    print("Aktora nie ma w danych treningowych i walidacyjnych. Wybierz innego aktora.")

,337799
nconst,nm0000204
tconst,"[tt0121766, tt1032825, tt8918080, tt1094249, t..."
category,actress
primaryName,Natalie Portman
knownForTitles,"[tt0947798, tt0376541, tt0110413, tt0434409]"
no_nominations_oscars_norm,0.068
no_nominations_globes_norm,0.174
no_nominations_emmy_norm,0.5
no_films_norm,0.006
average_films_rating_norm,0.666


Wyciągnięcie drugiego aktora

In [10]:
actor2_id = "nm0461136"

In [11]:
if (train_data['nconst'] == actor2_id).any():
    actor2_data = find_actor(train_data,  actor2_id)
    display(pd.DataFrame(actor2_data))
    actor2_data = prepare_pandas_row(actor2_data)
elif (validate_data['nconst'] == actor2_id).any():
    actor2_data = find_actor(validate_data,  actor2_id)
    display(pd.DataFrame(actor2_data))
    actor2_data = prepare_pandas_row(actor2_data)
else:
    print("Aktora nie ma w danych treningowych i walidacyjnych. Wybierz innego aktora.")

,993366
nconst,nm0461136
tconst,"[tt0368925, tt0366627, tt0409997, tt1307068, t..."
category,actress
primaryName,Keira Knightley
knownForTitles,"[tt0325980, tt2084970, tt0783233, tt0414387]"
no_nominations_oscars_norm,0.045
no_nominations_globes_norm,0.13
no_nominations_emmy_norm,0.5
no_films_norm,0.005
average_films_rating_norm,0.648


Wyniki similarity dla tych dwóch aktorów

In [13]:
similarity_value = similarity_new(actor1_data, actor2_data, True)
print("Wartość similarity dla ", actor1_data[3], " i ", actor2_data[3], " wynosi: ", similarity_value)

Wartość similarity dla  Natalie Portman  i  Keira Knightley  wynosi:  0.20127943573667717


Zapisanie porównywanych par aktorów wraz wartością similarity do pliku na Google Drive.

In [15]:
file_object = open(SIMILARITY_VALUES, 'a')
text = "{}, {}, similarity value: {}\n".format(actor1_data[3], actor2_data[3], similarity_value)
file_object.write(text)
file_object.close()

Zapisanie wniosków, jeśli wyszło coś ciekawego

# Ranking najbardziej podobnych aktorów do podanego

Wyciągnięcie aktora

Weź id aktora ze strony imdb. Wpisujemy imię i nazwisko aktora w wyszukiwarkę na imdb i kopiujemy jego id z linku, np. dla Natalie Portman id bierzemy z linku: https://www.imdb.com/name/nm0000204/?ref_=nv_sr_srsg_0

In [16]:
actor_id = "nm0000204"

In [17]:
if (train_data['nconst'] == actor_id).any():
    actor_data = find_actor(train_data,  actor_id)
    display(pd.DataFrame(actor_data))
    actor_data = prepare_pandas_row(actor_data)
elif (validate_data['nconst'] == actor_id).any():
    actor_data = find_actor(validate_data,  actor_id)
    display(pd.DataFrame(actor_data))
    actor_data = prepare_pandas_row(actor_data)
else:
    print("Aktora nie ma w danych treningowych i walidacyjnych. Wybierz innego aktora.")

,337799
nconst,nm0000204
tconst,"[tt0121766, tt1032825, tt8918080, tt1094249, t..."
category,actress
primaryName,Natalie Portman
knownForTitles,"[tt0947798, tt0376541, tt0110413, tt0434409]"
no_nominations_oscars_norm,0.068
no_nominations_globes_norm,0.174
no_nominations_emmy_norm,0.5
no_films_norm,0.006
average_films_rating_norm,0.666


Ranking 3 najbardziej podobnych aktorów do podanego powyżej

In [19]:
if (train_data['nconst'] == actor_id).any():
    ids, values = similarity_one_vs_all_new(train_data, actor_data, True)
    ids, values = select_top_similiar(ids, values, 3)
    names = replace_ids_with_names(train_data, ids, 3)
elif (validate_data['nconst'] == actor_id).any():
    ids, values = similarity_one_vs_all_new(validate_data, actor_data, True)
    ids, values = select_top_similiar(ids, values, 3)
    names = replace_ids_with_names(validate_data, ids, 3)
ranking = print_top_similiar(actor_data[3], names, values)

Najbardziej podobnymi do Natalie Portman aktorami/aktorkami są w kolejności:
  - Penélope Cruz z similarity równym: 0.284
  - Alicia Vikander z similarity równym: 0.283
  - Liv Ullmann z similarity równym: 0.281


Zapisanie do pliku

In [20]:
file_object = open(RANKINGS, 'a')
text = "Najbardziej podobni do {}:\n- {}, similarity: {}\n- {}, similarity: {}\n- {}, similarity: {}\n".format(actor_data[3], names[0], values[0], names[1], values[1], names[2], values[2])
file_object.write(text)
file_object.close()

Zapisanie wniosków, jeśli wyszło coś ciekawego